In [25]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################


username = "aacuser"
password = "testpass123"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
print(len(df.to_dict(orient='records')))
print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png'  # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())


#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), style={'height':'60px'}),
html.P("Created by Jess Dowd - June 2025", style={'textAlign': 'center'}),


app.layout = html.Div([
    html.Div(
        style={'display': 'flex', 'alignItems': 'center', 'justifyContent': 'flex-start', 'gap': '10px', 'padding': '0px'},
        children=[
            html.A(
                html.Img(
                    src='data:image/png;base64,{}'.format(encoded_image.decode()),
                    style={'height': '60px'}
                ),
                href='https://www.snhu.edu'
            ),
            html.Div([
                html.H1('Grazioso Salvare Search & Rescue Dog Dashboard', style={'fontSize': '24px'}),
                html.P("Created by Jess Dowd - June 2025", style={'marginTop': '-10px'})
            ])
        ]
    ),
    html.Hr(),

    html.Div(
    #FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'All', 'value': 'All'},
                {'label': 'Water Rescue', 'value': 'Water'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain or Wilderness'},
                {'label': 'Disaster or Individual Tracking', 'value': 'Disaster or Individual'}
            ],
            value='All',
            labelStyle={'display': 'inline-block', 'margin-right': '20px'}
        )

        
    ),
    html.Hr(),
        dash_table.DataTable(id='datatable-id',
            columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
            data=df.to_dict('records'),
            #FIXME: Set up the features for your interactive data table to make it user-friendly for your client
            #If you completed the Module Six Assignment, you can copy in the code you created here 
            row_selectable="single",
            selected_rows=[0],
            page_size=10,
            style_table={'overflowX': 'auto'}
        ),

    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(Output('datatable-id','data'),
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    ## FIX ME Add code to filter interactive data table with MongoDB queries
    if filter_type == 'Water':
        query = {
            'breed': {'$in': ['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland']},
            'sex_upon_outcome': 'Intact Female',
            'age_upon_outcome_in_weeks': {'$gte': 26, '$lte': 156}
        }
    elif filter_type == 'Mountain or Wilderness':
        query = {
            'breed': {'$in': ['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler']},
            'sex_upon_outcome': 'Intact Male',
            'age_upon_outcome_in_weeks': {'$gte': 26, '$lte': 156}
        }
    elif filter_type == 'Disaster or Individual':
        query = {
            'breed': {'$in': ['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler']},
            'sex_upon_outcome': 'Intact Male',
            'age_upon_outcome_in_weeks': {'$gte': 20, '$lte': 300}
        }
    else:
        query = {}

    filtered_df = pd.DataFrame.from_records(db.read(query))

    if '_id' in filtered_df.columns:
        filtered_df.drop(columns=['_id'], inplace=True)

  
    columns = [{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data = filtered_df.to_dict('records')

    return data



# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graphs(viewData):
    if viewData is None or len(viewData) == 0:
        return []

    dff = pd.DataFrame.from_dict(viewData)

    # collapse minor breeds into 'Other' if too many
    breed_counts = dff['breed'].value_counts()
    top_breeds = breed_counts.head(10).index
    dff['breed_grouped'] = dff['breed'].apply(lambda x: x if x in top_breeds else 'Other')

    fig = px.pie(
        dff,
        names='breed_grouped',
        title='Breed Distribution in Filtered Data',
        width=700,
        height=500
    )

    fig.update_layout(
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=-0.3,
            xanchor="center",
            x=0.5,
            font=dict(size=10)
        ),
        margin=dict(l=20, r=20, t=40, b=100)
    )

    return [dcc.Graph(figure=fig)]


    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if not selected_columns:
        return []
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]



# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]



app.run_server(debug=True)


9968
Index(['rec_num', 'age_upon_outcome', 'animal_id', 'animal_type', 'breed',
       'color', 'date_of_birth', 'datetime', 'monthyear', 'name',
       'outcome_subtype', 'outcome_type', 'sex_upon_outcome', 'location_lat',
       'location_long', 'age_upon_outcome_in_weeks'],
      dtype='object')
Dash app running on http://127.0.0.1:28145/
